# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

This (modified) notebook illustrates how to use DLC to:
- create a training set
- train a network
- evaluate a network
- create simple quality check plots
- analyze novel videos!

This shows the most simple code to do so, but many of the functions have additional features, so check out the overview & the protocol paper!

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.

Paper: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf

## Import DeepLabCut
This step assumes that you have installed DeepLabCut using the DLC-CPU.yaml conda config file. 

1. Install [Anaconda 3](https://www.anaconda.com/distribution/) if you don't have it already
2. Download DLC's [conda file for CPU](https://github.com/DeepLabCut/DeepLabCut/blob/master/conda-environments/DLC-CPU.yaml)
3. Open terminal where you placed the file and run `conda env create -f DLC-CPU.yaml`

In [ ]:
import deeplabcut

In [ ]:
# update the paths to your video files
# PLEASE EDIT THESE:
path = '/home/ryan/code/repos/dog-pose-estimation/videos/train'
deeplabcut.create_new_project(f'dlc-dog-pose','ryan', \
                              [f'{path}/coco_train01.mov', \
                               f'{path}/coco_train02.mov', \
                               f'{path}/coco_train03.mov', \
                               f'{path}/coco_train04.mov', \
                               f'{path}/coco_train05.mov', \
                               f'{path}/coco_train06.mov', \
                               f'{path}/coco_train07.mov', \
                               f'{path}/coco_train08.mov', \
                               f'{path}/coco_train09.mov', \
                               f'{path}/coco_train10.mov'],
              copy_videos=True, multianimal=False)

In [ ]:
# Setup your project variables:
# PLEASE EDIT THESE:
ProjectFolderName = 'dlc-dog-pose'
VideoType = 'mov' 

In [ ]:
# This creates a path variable to your project's config.yaml file
path_config_file = '/home/ryan/code/repos/dog-pose-estimation/dlc-dog-pose-ryan-2021-02-14/config.yaml'
path_config_file

### Edit the config.yaml file to add body parts. I labeled these parts of my dog:
```
bodyparts:
- nose
- leftear
- rightear
- collar
- spine1
- spine2
- spine3
- tailstart
- tailend
- frontleftleg
- frontleftpaw
- frontrightleg
- frontrightpaw
- backleftleg
- backleftpaw
- backrightleg
- backrightpaw
```

In [ ]:
# This step is to extract frames from the videos you added to the project
deeplabcut.extract_frames(path_config_file, mode='automatic', algo='kmeans', crop=True)

In [ ]:
# Label your frames
# IMPORTANT: You must run this step from a computer with a display or use VNC (GUI required)
deeplabcut.label_frames('./config.yaml')

## Create a training dataset:
After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [ ]:
# There are many more functions you can set here, including which network you use
# check the docstring for full options you can use
deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug')

## Start training:
This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
# Typically, you want to train to 200,000 + iterations.
# more info and there are more things you can set: https://github.com/AlexEMG/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network

deeplabcut.train_network(path_config_file, shuffle=1, displayiters=100,saveiters=500)

# This will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.03M iterations). 
# Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

**When you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating:
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [ ]:
%matplotlib notebook
deeplabcut.evaluate_network(path_config_file,plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, 
#so be sure your labels are good! (And you have trained enough ;)

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:
# This is the location of the videos to analyze
videofile_path = ['/home/ryan/code/repos/dog-pose-estimation/videos']
videofile_path

In [ ]:
deeplabcut.analyze_videos(path_config_file,videofile_path, videotype='mov')

## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
deeplabcut.plot_trajectories(path_config_file,videofile_path, videotype='mov')

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4. 

## Create labeled video:
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(path_config_file, videofile_path, videotype='mov', draw_skeleton=True)

In [ ]:
# PLACEHOLDER: This step is where we will convert to OpenVINO for faster inference 
# export frozen TF graph (.pb format)
deeplabcut.export_model(config_path, iteration=None, shuffle=1, trainingsetindex=0, snapshotindex=None, TFGPUinference=False, overwrite=False, make_tar=True)